In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from matplotlib import pyplot as plt

from sklearn import model_selection as ms
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier

import xgboost as xgb

import gzip

import import_ipynb
from My_Functions import null_cols, rmse

importing Jupyter notebook from My_Functions.ipynb


In [2]:
acc_nonorm = pd.read_csv("acc_nonorm_touse.csv.gz",
                        compression='gzip',
                        header=0,
                        sep=',',
                        quotechar='"')

acc_partialnorm = pd.read_csv("acc_partialnorm_touse.csv.gz",
                        compression='gzip',
                        header=0,
                        sep=',',
                        quotechar='"')

acc_fullnorm = pd.read_csv("acc_fullnorm_touse.csv.gz",
                        compression='gzip',
                        header=0,
                        sep=',',
                        quotechar='"')

target_touse = pd.read_csv("target_touse.csv.gz",
                           compression='gzip',
                           header=0,
                           sep=',',
                           quotechar='"')

# acc_forprediction = pd.read_csv("acc_forprediction.csv.gz",
#                                 compression='gzip',
#                                 header=0,
#                                 sep=',',
#                                 quotechar='"')

# target_forprediction = pd.read_csv("target_forprediction.csv.gz",
#                                    compression='gzip',
#                                    header=0,
#                                    sep=',',
#                                    quotechar='"')

In [3]:
state=0

# nonorm:
xtrain_no, xtest_no, ytrain_no, ytest_no = ms.train_test_split(acc_nonorm,
                                                               target_touse,
                                                               test_size=0.2,
                                                               random_state=state)

# Must flatten to fit
ytrain_no = ytrain_no.values.flatten()


# partialnorm:
xtrain_partial, xtest_partial, ytrain_partial, ytest_partial = ms.train_test_split(acc_partialnorm,
                                                                                   target_touse,
                                                                                   test_size=0.2,
                                                                                   random_state=state)

# Must flatten to fit
ytrain_partial = ytrain_partial.values.flatten()


# fullnorm:
xtrain_full, xtest_full, ytrain_full, ytest_full = ms.train_test_split(acc_fullnorm,
                                                                       target_touse,
                                                                       test_size=0.2,
                                                                       random_state=state)

# Must flatten to fit
ytrain_full = ytrain_full.values.flatten()


# using standard kfold split
n_folds = ms.KFold(n_splits=5, random_state=state, shuffle=True)

In [ ]:
## Liblinear

In [ ]:
# logistic = LogisticRegression()

# train_scores = []
# test_scores = []
# train_rmse = []
# test_rmse = []
# best_par_list = []

# penalties = ['l1', 'l2']
# # duals = [False]
# Cs = [0.001, 0.1, 1]
# fitintercept = [True, False]
# # classweight = [{'grade':2, 'sub_grade': 2}]
# randomstate = [state]
# solvers = ['liblinear']
# # max_iters = [1e4]
# verbose_ = [1]
# warmstart = [True, False]
# # njobs = [-1]
# # l1_ratios = [0, 0.5, 1]

# gparam_liblin = {'penalty': penalties,
# #              'dual': duals,
#              'C': Cs,
#              'fit_intercept': fitintercept,
# #              'class_weight': classweight,
#              'random_state': randomstate,
#              'solver': solvers,
# #              'max_iter': max_iters,
#              'verbose': verbose_,
#              'warm_start': warmstart}
# #              'n_jobs': njobs}
# #           'l1_ratio': l1_ratios}


# gs_liblin = ms.GridSearchCV(logistic, gparam_liblin, cv=n_folds, refit=True, n_jobs=-1,
#                         scoring='neg_root_mean_squared_error', return_train_score=True)

# %time gs_liblin.fit(xtrain, ytrain)

# # setting up dataframe for results
# train_scores.append(gs_liblin.best_estimator_.score(xtrain, ytrain))
# test_scores.append(gs_liblin.best_estimator_.score(xtest, ytest))

# # use rmse function from Self_Written_Functions_Sheet_Recover
# train_rmse.append(rmse(gs_liblin, ytrain, xtrain))
# test_rmse.append(rmse(gs_liblin, ytest, xtest))

# # add the best parameters to the df
# best_par_list.append(gs_liblin.best_params_)

# # find the difference btwn the rmses
# diff_rmse = np.subtract(train_rmse, test_rmse)

# # create dataframe
# list_results = [train_scores, test_scores, train_rmse, test_rmse, diff_rmse]
# res_df = pd.DataFrame(list_results).T
# res_df.columns = ['TrainScores', 'TestScores', 'TrainRMSE', 'TestRMSE', 'DiffRMSE']
# best_par_df = pd.DataFrame(best_par_list)
# res_df = pd.concat([res_df, best_par_df], axis=1, sort=False)

# res_df

1


In [ ]:
# confusion_matrix(ytrain, gs_liblin.predict(xtrain))

In [ ]:
# confusion_matrix(ytest, gs_liblin.predict(xtest))

## SAGA

In [6]:
logistic = LogisticRegression()

train_scores = []
test_scores = []
train_rmse = []
test_rmse = []
best_par_list = []

penalties = ['l1', 'l2', 'elasticnet', 'none']
Cs = [0.001, 0.1, 1]
randomstate = [state]
solvers = ['liblinear']
# max_iters = [1e4]
verbose_ = [1]
warmstart = [True, False]
# njobs = [-1]
# l1_ratios = [0, 0.5, 1]

gparam_saga = {'penalty': penalties,
               'C': Cs,
               'random_state': randomstate,
               'solver': solvers,
#              'max_iter': max_iters,
               'verbose': verbose_,
               'warm_start': warmstart}
#              'n_jobs': njobs}
#           'l1_ratio': l1_ratios}


gs_saga = ms.GridSearchCV(logistic, gparam_saga, cv=n_folds, refit=True, n_jobs=-1,
                          scoring='neg_root_mean_squared_error', return_train_score=True)

%time gs_saga.fit(xtrain_full, ytrain_full)

# setting up dataframe for results
train_scores.append(gs_saga.best_estimator_.score(xtrain_full, ytrain_full))
test_scores.append(gs_saga.best_estimator_.score(xtest_full, ytest_full))


# use rmse function from Self_Written_Functions_Sheet_Recover
train_rmse.append(rmse(gs_saga, ytrain_full, xtrain_full))
test_rmse.append(rmse(gs_saga, ytest_full, xtest_full))

# add the best parameters to the df
best_par_list.append(gs_saga.best_params_)

# find the difference btwn the rmses
diff_rmse = np.subtract(train_rmse, test_rmse)

# create dataframe
list_results = [train_scores, test_scores, train_rmse, test_rmse, diff_rmse]
res_df = pd.DataFrame(list_results).T
res_df.columns = ['TrainScores', 'TestScores', 'TrainRMSE', 'TestRMSE', 'DiffRMSE']
best_par_df = pd.DataFrame(best_par_list)
res_df = pd.concat([res_df, best_par_df], axis=1, sort=False)

res_df

C:\Users\tdcho\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [-0.59731753 -0.59731753 -0.59214414 -0.59214414         nan         nan
         nan         nan -0.58413564 -0.58413564 -0.5849297  -0.5849297
         nan         nan         nan         nan -0.58280194 -0.58280194
 -0.58320017 -0.58320017         nan         nan         nan         nan]
  warnings.warn(
C:\Users\tdcho\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the train scores are non-finite: [-0.59718599 -0.59718599 -0.59184988 -0.59184988         nan         nan
         nan         nan -0.5834465  -0.5834465  -0.58444387 -0.58444387
         nan         nan         nan         nan -0.58190333 -0.58190333
 -0.58241049 -0.58241049         nan         nan         nan         nan]
  warnings.warn(


[LibLinear]Wall time: 24min 40s


,TrainScores,TestScores,TrainRMSE,TestRMSE,DiffRMSE,C,penalty,random_state,solver,verbose,warm_start
0,0.661169,0.662069,0.582092,0.581318,0.000774,1,l1,0,liblinear,1,True


In [7]:
confusion_matrix(ytrain_full, gs_saga.predict(xtrain_full))

array([[71675, 36035],
       [36979, 70799]], dtype=int64)

In [8]:
confusion_matrix(ytest_full, gs_saga.predict(xtest_full))

array([[17917,  9053],
       [ 9152, 17750]], dtype=int64)

## SAG

In [9]:
logistic = LogisticRegression()

train_scores = []
test_scores = []
train_rmse = []
test_rmse = []
best_par_list = []

penalties = ['l2', 'none']
# duals = [False]
Cs = [0.001, 0.1, 1]
fitintercept = [True, False]
# classweight = [{'grade':2, 'sub_grade': 2}]
randomstate = [state]
solvers = ['sag']
# max_iters = [1e4]
verbose_ = [1]
warmstart = [True, False]
# njobs = [-1]
# l1_ratios = [0, 0.5, 1]

gparam_sag = {'penalty': penalties,
#              'dual': duals,
              'C': Cs,
              'fit_intercept': fitintercept,
#              'class_weight': classweight,
              'random_state': randomstate,
              'solver': solvers,
#              'max_iter': max_iters,
              'verbose': verbose_,
              'warm_start': warmstart}
#              'n_jobs': njobs}
#           'l1_ratio': l1_ratios}


gs_sag = ms.GridSearchCV(logistic, gparam_sag, cv=n_folds, refit=True, n_jobs=-1,
                         scoring='neg_root_mean_squared_error', return_train_score=True)

%time gs_sag.fit(xtrain_full, ytrain_full)

# setting up dataframe for results
train_scores.append(gs_sag.best_estimator_.score(xtrain_full, ytrain_full))
test_scores.append(gs_sag.best_estimator_.score(xtest_full, ytest_full))

# use rmse function from Self_Written_Functions_Sheet_Recover
train_rmse.append(rmse(gs_sag, ytrain_full, xtrain_full))
test_rmse.append(rmse(gs_sag, ytest_full, xtest_full))

# add the best parameters to the df
best_par_list.append(gs_sag.best_params_)

# find the difference btwn the rmses
diff_rmse = np.subtract(train_rmse, test_rmse)

# create dataframe
list_results = [train_scores, test_scores, train_rmse, test_rmse, diff_rmse]
res_df = pd.DataFrame(list_results).T
res_df.columns = ['TrainScores', 'TestScores', 'TrainRMSE', 'TestRMSE', 'DiffRMSE']
best_par_df = pd.DataFrame(best_par_list)
res_df = pd.concat([res_df, best_par_df], axis=1, sort=False)

res_df

C:\Users\tdcho\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 30 seconds
Wall time: 2min 44s


C:\Users\tdcho\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   29.8s finished


,TrainScores,TestScores,TrainRMSE,TestRMSE,DiffRMSE,C,fit_intercept,penalty,random_state,solver,verbose,warm_start
0,0.661401,0.662682,0.581893,0.580791,0.001102,0.001,True,none,0,sag,1,True


In [10]:
confusion_matrix(ytrain_full, gs_sag.predict(xtrain_full))

array([[71695, 36015],
       [36949, 70829]], dtype=int64)

In [11]:
confusion_matrix(ytrain_full, gs_sag.predict(xtrain_full))

array([[71695, 36015],
       [36949, 70829]], dtype=int64)

# XGBoost

### No Normalization

In [13]:
xgb_clsfr = xgb.XGBClassifier(objective='binary:logistic', random_state=state,
                              eval_metric='rmse', use_label_encoder=False,
#                               tree_method='gpu_hist', predictor='gpu_predictor',
                              gamma=0.01)

train_scores = []
test_scores = []
train_rmse = []
test_rmse = []
best_par_list = []

nestimators = [1, 5, 15, 50]
booster_ = ['gbtree', 'gblinear']
maxdepth = [1, 5, 15, 50]
learningrate = [0.01, 0.1, 1]
# gamma_ = [0.01, 0.05, 0.1, 0.5, 1]

gparam_xgb = {'n_estimators': nestimators,
              'booster': booster_,
              'max_depth': maxdepth,
              'learning_rate': learningrate}
#               'gamma': gamma_}

gs_xgb = ms.GridSearchCV(xgb_clsfr, gparam_xgb, cv=n_folds, refit=True, n_jobs=-1,
                         scoring='neg_root_mean_squared_error', return_train_score=True)

%time gs_xgb.fit(xtrain_no, ytrain_no)

# setting up dataframe for results
train_scores.append(gs_xgb.best_estimator_.score(xtrain_no, ytrain_no))
test_scores.append(gs_xgb.best_estimator_.score(xtest_no, ytest_no))

# use rmse function from Self_Written_Functions_Sheet_Recover
train_rmse.append(rmse(gs_xgb, ytrain_no, xtrain_no))
test_rmse.append(rmse(gs_xgb, ytest_no, xtest_no))

# add the best parameters to the df
best_par_list.append(gs_xgb.best_params_)

# find the difference btwn the rmses
diff_rmse = np.subtract(train_rmse, test_rmse)

# create dataframe
list_results = [train_scores, test_scores, train_rmse, test_rmse, diff_rmse]
res_df = pd.DataFrame(list_results).T
res_df.columns = ['TrainScores', 'TestScores', 'TrainRMSE', 'TestRMSE', 'DiffRMSE']
best_par_df = pd.DataFrame(best_par_list)
res_df = pd.concat([res_df, best_par_df], axis=1, sort=False)

res_df

C:\Users\tdcho\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [        nan         nan         nan         nan -0.59742426         nan
         nan -0.59254665 -0.61828033 -0.61194485 -0.60212223 -0.59239746
 -0.64819536 -0.63392775 -0.62085013 -0.60483663 -0.60335671 -0.60335671
 -0.60055912 -0.59386847 -0.59742426 -0.59192716 -0.58785714 -0.57957793
 -0.61828033 -0.59977032 -0.59070528 -0.58404932 -0.64819536 -0.61379979
 -0.59853984 -0.58790989 -0.60335671 -0.60073654 -0.59195436 -0.58440125
 -0.59742426 -0.58761678 -0.5838878  -0.58689947 -0.61828033 -0.62115722
 -0.62473824 -0.61771136 -0.64819536 -0.62719687 -0.61674986 -0.61004085
 -0.62232634 -0.61493107 -0.60678408 -0.59440126 -0.62324484 -0.61478382
 -0.60685664 -0.5943896  -0.6216565  -0.61487439 -0.60684533 -0.59436221
 -0.62420753 -0.61503296 -0.60673047 -0.59437776 -0.61332828 -0.5947054
 -0.58802726 -0.58525932 -0.61455328 -0.59458082 -0.587

Wall time: 15min 45s


,TrainScores,TestScores,TrainRMSE,TestRMSE,DiffRMSE,booster,learning_rate,max_depth,n_estimators
0,0.670144,0.664742,0.574331,0.579014,-0.004684,gbtree,0.1,5,50


In [14]:
confusion_matrix(ytrain_no, gs_xgb.predict(xtrain_no))

array([[70144, 37566],
       [33514, 74264]], dtype=int64)

In [15]:
confusion_matrix(ytest_no, gs_xgb.predict(xtest_no))

array([[17349,  9621],
       [ 8440, 18462]], dtype=int64)

In [16]:
sorted_importance = sorted(zip(xtrain_no.columns,
                               gs_xgb.best_estimator_.feature_importances_),
                           key=lambda t:t[1], reverse=True)
sorted_importance

[('grade', 0.42365357),
 ('sub_grade', 0.17668334),
 ('term', 0.018636014),
 ('open_rv_24m', 0.018029213),
 ('home_ownership_RENT', 0.012181467),
 ('acc_open_past_24mths', 0.010916078),
 ('int_rate', 0.010462329),
 ('year', 0.010359949),
 ('dti', 0.009659388),
 ('avg_cur_bal', 0.008983572),
 ('all_util', 0.008613097),
 ('purpose_small_business', 0.008607892),
 ('mort_acc', 0.008442886),
 ('num_tl_120dpd_2m', 0.008056527),
 ('emp_length', 0.008012441),
 ('fico_range_low', 0.007887046),
 ('annual_inc', 0.0076512094),
 ('num_actv_rev_tl', 0.0073627904),
 ('month', 0.006974463),
 ('addr_state_WA', 0.006893026),
 ('addr_state_SC', 0.0068826987),
 ('home_ownership_MORTGAGE', 0.0068597193),
 ('mths_since_recent_inq', 0.0061105555),
 ('installment', 0.0061016087),
 ('addr_state_OR', 0.006045591),
 ('addr_state_CO', 0.0060269632),
 ('tot_hi_cred_lim', 0.005620666),
 ('sec_app_revol_util', 0.0050641466),
 ('funded_amnt', 0.004975227),
 ('delinq_2yrs', 0.0049588517),
 ('max_bal_bc', 0.0047628502)

### Partial Normalization

In [ ]:
xgb_clsfr = xgb.XGBClassifier(objective='binary:logistic', random_state=state,
                              eval_metric='rmse', use_label_encoder=False,
                              tree_method='gpu_hist', predictor='gpu_predictor',
                              gamma=0.01)

train_scores = []
test_scores = []
train_rmse = []
test_rmse = []
best_par_list = []

nestimators = [1, 5, 15, 50]
booster_ = ['gbtree', 'gblinear']
maxdepth = [1, 5, 15, 50]
learningrate = [0.01, 0.1, 1]
# gamma_ = [0.01, 0.05, 0.1, 0.5, 1]

gparam_xgb = {'n_estimators': nestimators,
              'booster': booster_,
              'max_depth': maxdepth,
              'learning_rate': learningrate}
#               'gamma': gamma_}

gs_xgb = ms.GridSearchCV(xgb_clsfr, gparam_xgb, cv=n_folds, refit=True, n_jobs=-1,
                         scoring='neg_root_mean_squared_error', return_train_score=True)

%time gs_xgb.fit(xtrain_partial, ytrain_partial)

# setting up dataframe for results
train_scores.append(gs_xgb.best_estimator_.score(xtrain_partial, ytrain_partial))
test_scores.append(gs_xgb.best_estimator_.score(xtest_partial, ytest_partial))

# use rmse function from Self_Written_Functions_Sheet_Recover
train_rmse.append(rmse(gs_xgb, ytrain_partial, xtrain_partial))
test_rmse.append(rmse(gs_xgb, ytest_partial, xtest_partial))

# add the best parameters to the df
best_par_list.append(gs_xgb.best_params_)

# find the difference btwn the rmses
diff_rmse = np.subtract(train_rmse, test_rmse)

# create dataframe
list_results = [train_scores, test_scores, train_rmse, test_rmse, diff_rmse]
res_df = pd.DataFrame(list_results).T
res_df.columns = ['TrainScores', 'TestScores', 'TrainRMSE', 'TestRMSE', 'DiffRMSE']
best_par_df = pd.DataFrame(best_par_list)
res_df = pd.concat([res_df, best_par_df], axis=1, sort=False)

res_df

In [ ]:
confusion_matrix(ytrain_partial, gs_xgb.predict(xtrain_partial))

In [ ]:
confusion_matrix(ytest_partial, gs_xgb.predict(xtest_partial))

In [ ]:
sorted_importance = sorted(zip(xtrain_partial.columns,
                               gs_xgb.best_estimator_.feature_importances_),
                           key=lambda t:t[1], reverse=True)
sorted_importance

### Full Normalization

In [ ]:
xgb_clsfr = xgb.XGBClassifier(objective='binary:logistic', random_state=state,
                              eval_metric='rmse', use_label_encoder=False,
                              tree_method='gpu_hist', predictor='gpu_predictor',
                              gamma=0.01)

train_scores = []
test_scores = []
train_rmse = []
test_rmse = []
best_par_list = []

nestimators = [1, 5, 15, 50]
booster_ = ['gbtree', 'gblinear']
maxdepth = [1, 5, 15, 50]
learningrate = [0.01, 0.1, 1]
# gamma_ = [0.01, 0.05, 0.1, 0.5, 1]

gparam_xgb = {'n_estimators': nestimators,
              'booster': booster_,
              'max_depth': maxdepth,
              'learning_rate': learningrate}
#               'gamma': gamma_}

gs_xgb = ms.GridSearchCV(xgb_clsfr, gparam_xgb, cv=n_folds, refit=True, n_jobs=-1,
                         scoring='neg_root_mean_squared_error', return_train_score=True)

%time gs_xgb.fit(xtrain_full, ytrain_full)

# setting up dataframe for results
train_scores.append(gs_xgb.best_estimator_.score(xtrain_full, ytrain_full))
test_scores.append(gs_xgb.best_estimator_.score(xtest_full, ytest_full))

# use rmse function from Self_Written_Functions_Sheet_Recover
train_rmse.append(rmse(gs_xgb, ytrain_full, xtrain_full))
test_rmse.append(rmse(gs_xgb, ytest_full, xtest_full))

# add the best parameters to the df
best_par_list.append(gs_xgb.best_params_)

# find the difference btwn the rmses
diff_rmse = np.subtract(train_rmse, test_rmse)

# create dataframe
list_results = [train_scores, test_scores, train_rmse, test_rmse, diff_rmse]
res_df = pd.DataFrame(list_results).T
res_df.columns = ['TrainScores', 'TestScores', 'TrainRMSE', 'TestRMSE', 'DiffRMSE']
best_par_df = pd.DataFrame(best_par_list)
res_df = pd.concat([res_df, best_par_df], axis=1, sort=False)

res_df

In [ ]:
confusion_matrix(ytrain_full, gs_xgb.predict(xtrain_full))

In [ ]:
confusion_matrix(ytest_full, gs_xgb.predict(xtest_full))

In [ ]:
sorted_importance = sorted(zip(xtrain_full.columns,
                               gs_xgb.best_estimator_.feature_importances_),
                           key=lambda t:t[1], reverse=True)
sorted_importance